In [0]:
import tensorflow as tf
from collections import Counter
import numpy as np

from google.colab import files
from google.colab import files

In [2]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sentences.train to sentences.train
User uploaded file "sentences.train" with length 111668947 bytes


### **1) PREPROCESSING**

In [0]:
def _sentence_preprocessing(sentence):
    sentence = ['<bos>'] + line.split() + ['<eos>']
    if len(sentence) <= 30:
        sentence = sentence + ['<pad>']*(30-len(sentence))
        return sentence
    #we ignore sentences with more than 30 words/tokens
    return []

In [0]:
tokens = []

with open("sentences.train", "r") as file:
    for line in file:
        tokens.extend(_sentence_preprocessing(line))
        
file.close()

In [5]:
#build vocab of 20k most frequent words
vocab = Counter(tokens)
vocab_20k = list(map(lambda x: x[0], vocab.most_common(20000)))
vocab_20k[:10]

['<pad>', '<bos>', '<eos>', '.', ',', 'the', 'i', 'to', 'and', '``']

In [6]:
word_list = list(set(tokens))
len(word_list)

175268

In [0]:
#build words to ids dictionary
word_to_id_dict = dict({'<bos>': 1, '<eos>': 2, '<pad>': 3, '<unk>': 4})
id_counter = 5 
for word in word_list:
    if word_to_id_dict.get(word, 0) == 0 and word in vocab_20k:
        word_to_id_dict[word] = id_counter
        id_counter += 1

In [0]:
words_ids = np.array([word_to_id_dict.get(word,4) for word in tokens]).reshape(int(len(tokens)/30), 30)

In [9]:
words_ids[0,:]

array([    1, 15222, 17070,  3296,  9077, 12074, 19453,     4,  1706,
       12457, 18797,  5159,  3697,  9077,   466, 13211, 13429, 19453,
        1201,    25,     2,     3,     3,     3,     3,     3,     3,
           3,     3,     3])

In [0]:
#input_tensor = tf.convert_to_tensor(words_ids)
#input_data = tf.data.Dataset.from_tensors(input_tensor)

### *2) DATASET API*

In [0]:
def parse(line):
    line_split = tf.string_split([line])
    input_seq = tf.string_to_number(line_split.values[:-1], out_type=tf.int32)
    output_seq = tf.string_to_number(line_split.values[1:], out_type=tf.int32)
    return input_seq, output_seq

In [0]:
batch_size = 64
hidden_units = 512

file_name_train = tf.placeholder(tf.string)

training_dataset = tf.data.TextLineDataset(file_name_train).map(parse).batch(batch_size)
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
input_batch, output_batch = iterator.get_next()
training_init_op = iterator.make_initializer(training_dataset)

#init_state = tf.placeholder(tf.float32, [batch_size, hidden_units])
#cell = tf.nn.rnn_cell.LSTMCell(hidden_units, )

#current_words = tf.placeholder(tf.float32, [batch_size, None])
#output, new_state = tf.map_fn(cell,elems=[(current_words, init_state)])  #this does not work...

In [44]:
#print out first three batches
with tf.Session() as sess:
    sess.run(training_init_op, {file_name_train: "train.ids"})
    sess.run(tf.initialize_all_variables())
    for _ in range(3):
        a, b = sess.run([input_batch, output_batch])
        print(a)
        print(b)
        print(a.shape)
        print(b.shape)
    
        #c, d = sess.run([output, new_state], {current_words = a[:,0]})
        #print(c)
        #print(d)
        #print(c.shape)
        #print(d.shape)

[[    1 15222 17070 ...     3     3     3]
 [    1 15222 16990 ...     3     3     3]
 [    1     4  3631 ...     3     3     3]
 ...
 [    1  2099 15009 ...     3     3     3]
 [    1 17070  9116 ...     3     3     3]
 [    1 15222 10077 ...     3     3     3]]
[[15222 17070  3296 ...     3     3     3]
 [15222 16990   444 ...     3     3     3]
 [    4  3631  3481 ...     3     3     3]
 ...
 [ 2099 15009  8411 ...     3     3     3]
 [17070  9116 13684 ...     3     3     3]
 [15222 10077 18797 ...     3     3     3]]
(64, 29)
(64, 29)
[[    1  5587  8218 ...     3     3     3]
 [    1  6313  4419 ...     3     3     3]
 [    1  7297     4 ...     3     3     3]
 ...
 [    1  8246  8048 ...     3     3     3]
 [    1  1055  2099 ...     3     3     3]
 [    1 15222 18758 ...     3     3     3]]
[[ 5587  8218 18797 ...     3     3     3]
 [ 6313  4419 13926 ...     3     3     3]
 [ 7297     4  7836 ...     3     3     3]
 ...
 [ 8246  8048 11177 ...     3     3     3]
 [ 1055  2099

### **3) TENSORFLOW PRACTICE/MESS**

In [0]:
with open('train.ids', 'w') as file:
  for i in range(words_ids.shape[0]):
    file.write(' '.join(str(x) for x in list(words_ids[i,:])) + '\n')
    
file.close()

In [0]:
def parse(line):
  line_split = tf.string_split([line])
  input_seq = tf.string_to_number(line_split.values[:-1], out_type=tf.int32)
  output_seq = tf.string_to_number(line_split.values[1:], out_type=tf.int32)
  return input_seq, output_seq

In [19]:
batch_size = 64
vocab_size = 20000
embed_size = 100
hidden_units = 512
num_epochs = 2

file_name_train = tf.placeholder(tf.string)

training_dataset = tf.data.TextLineDataset(file_name_train).map(parse).batch(batch_size)
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
input_batch, output_batch = iterator.get_next()
training_init_op = iterator.make_initializer(training_dataset) 



# input_embedding_mat = tf.Variable(tf.truncated_normal([vocab_size, embed_size], stddev=0.12))
# output_embedding_mat = tf.Variable(tf.truncated_normal([vocab_size, hidden_units], stddev=0.12))

# #cell = tf.nn.rnn_cell.LSTMCell(hidden_units, reuse=True)
# #init_state = tf.placeholder(tf.float32, [batch_size, hidden_units])

# input_embedded = tf.nn.embedding_lookup(input_embedding_mat, input_batch)                        

# #TODO: substitue dynamic_rnn with for loop
# # outputs, _ = tf.nn.dynamic_rnn(
# #             cell=cell,
# #             inputs=input_embedded,
# #             dtype=tf.float32
# #         )


# def output_embedding(current_output):
#     return  tf.matmul(current_output, tf.transpose(output_embedding_mat))

# # To compute the logits
# logits = tf.map_fn(output_embedding, outputs)
# logits = tf.reshape(logits, [-1, vocab_size])
# loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.reshape(output_batch, [-1]), logits=logits) 


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    epoch = 0
    
    while epoch < num_epochs:

      sess.run(training_init_op, {file_name_train: "train.ids"})
      train_loss = 0.0
   
      while True:

          try:
            #_loss  = sess.run(loss)
            a = sess.run(input_batch)
            b = sess.run(output_batch)
            print(a)
            print(b)
            print(a.shape)
            print(b.shape)
            break
            
            #train_loss += _loss
            
         

          except tf.errors.OutOfRangeError:
              # The end of one epoch
              break

              
                                       
#opt = tf.train.AdagradOptimizer()
#gradients = tf.gradients(loss, params, colocate_gradients_with_ops=True)
#clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5)
#updates = opt.apply_gradients(zip(clipped_gradients, params), global_step=self.global_step)



[[    1 15222 17070 ...     3     3     3]
 [    1 15222 16990 ...     3     3     3]
 [    1     4  3631 ...     3     3     3]
 ...
 [    1  2099 15009 ...     3     3     3]
 [    1 17070  9116 ...     3     3     3]
 [    1 15222 10077 ...     3     3     3]]
[[ 5587  8218 18797 ...     3     3     3]
 [ 6313  4419 13926 ...     3     3     3]
 [ 7297     4  7836 ...     3     3     3]
 ...
 [ 8246  8048 11177 ...     3     3     3]
 [ 1055  2099     4 ...     3     3     3]
 [15222 18758  1201 ...     3     3     3]]
(64, 29)
(64, 29)
[[    1  2099 18348 ...     3     3     3]
 [    1  1147  7510 ...     3     3     3]
 [    1     4 17041 ...     3     3     3]
 ...
 [    1 15222 13101 ...     3     3     3]
 [    1  2099   532 ...     3     3     3]
 [    1     4 18797 ...     3     3     3]]
[[ 2099  5044 15738 ...     3     3     3]
 [17312 13211  9344 ...     3     3     3]
 [19815  3021  1706 ...     3     3     3]
 ...
 [15222 10592 17555 ...     3     3     3]
 [ 1704 12146

KeyboardInterrupt: ignored

In [0]:
train_loss

array([1336.4052, 1337.0669, 1336.8267, ..., 1341.0707, 1341.2993,
       1341.4226], dtype=float32)

In [0]:
type(train_loss)

numpy.ndarray

In [0]:
train_loss.shape

(1856,)